In [ ]:
import k3d
import numpy as np

import sys
sys.path.insert(0, "..")

from TimingCalib.MPMT import MPMT
from TimingCalib.PMT import PMT
from TimingCalib.LED import LED
from TimingCalib.WCD import WCD

from scipy.spatial.transform import Rotation as R

In [ ]:
wcte = WCD(None, 'WCTE', {}, {})

In [ ]:
# Show numbering and orientation of all mPMTs in the WCTE detector

plot = k3d.plot()

# mpmt coordinates
color_z = 0x0000ff
color_x = 0xff0000
color_y = 0x00ff00
origins = []
z_vecs = []
x_vecs = []
y_vecs = []
vec_length = 100. # length of x,z vectors to show for each pmt

color_mpmt = 0xabb2b9
n_point_mpmt = 8
indices_mpmt = []
for i in range(n_point_mpmt):
    indices_mpmt.append([i,(i+1)%n_point_mpmt])

n_point_ft = 20
indices_ft = []
for i in range(n_point_ft):
    indices_ft.append([i,(i+1)%n_point_ft])
    
# draw the extent of the mpmt baseplates
for i_mpmt,mpmt in enumerate(wcte.mpmts):
    
    location, direction_x, direction_z = mpmt.get_orientation('design')
    direction_y = np.cross(direction_z,direction_x)
    # lists to show mPMT coordinate systems
    z_vec = list(np.array(direction_z)*vec_length)
    x_vec = list(np.array(direction_x)*vec_length)
    y_vec = list(np.array(direction_y)*vec_length)
    origins.append(location)
    z_vecs.append(z_vec)
    x_vecs.append(x_vec)
    y_vecs.append(y_vec)
    
    baseplate_points = np.array(mpmt.get_xy_points('design'),dtype=np.float32)
    plt_baseplate = k3d.lines(baseplate_points, indices_mpmt, indices_type='segment', color=color_mpmt)
    plot += plt_baseplate
    
    feedthrough_points = np.array(mpmt.get_xy_points('design', feature='feedthrough'),dtype=np.float32)
    plt_feedthrough = k3d.lines(feedthrough_points, indices_ft, indices_type='segment', color=color_mpmt)
    plot += plt_feedthrough
    
    # k3d complains about the following not being float32!
    plt_text = k3d.text(str(i_mpmt), position=location, reference_point='cc', size=1., label_box=False, color=color_mpmt)
    plot += plt_text
    
plt_z_vecs = k3d.vectors(origins=origins, vectors=z_vecs, color=color_z, head_size=250.)
plot += plt_z_vecs
plt_x_vecs = k3d.vectors(origins=origins, vectors=x_vecs, color=color_x, head_size=250.)
plot += plt_x_vecs
plt_y_vecs = k3d.vectors(origins=origins, vectors=y_vecs, color=color_y, head_size=250.)
plot += plt_y_vecs

plot.display()

In [ ]:
# Show all mPMTs and some PMTs+LEDs in the WCTE detector

plot = k3d.plot()

# mpmt coordinates
color_z = 0x0000ff
color_x = 0xff0000
color_y = 0x00ff00
origins = []
z_vecs = []
x_vecs = []
vec_length = 100. # length of x,z vectors to show for each pmt

color_mpmt = 0xabb2b9
n_point_mpmt = 8
indices_mpmt = []
for i in range(n_point_mpmt):
    indices_mpmt.append([i,(i+1)%n_point_mpmt])
    
n_point_ft = 20
indices_ft = []
for i in range(n_point_ft):
    indices_ft.append([i,(i+1)%n_point_ft])
    
# pmt faces:
color_pmt = 0xdc7633
n_point_pmt = 20
indices_pmt = []
for i in range(n_point_pmt):
    indices_pmt.append([i,(i+1)%n_point_pmt])
    
# led coordinates
led_origins = []
led_z_vecs = []
led_vec_length = 30. # length of z vectors to show for each led

color_led = 0x00ff00
    
# draw the extent of the mpmt baseplates
for i_mpmt,mpmt in enumerate(wcte.mpmts):
    
    location, direction_x, direction_z = mpmt.get_orientation('design')
    # lists to show mPMT coordinate systems
    z_vec = list(np.array(direction_z)*vec_length)
    x_vec = list(np.array(direction_x)*vec_length)
    origins.append(location)
    z_vecs.append(z_vec)
    x_vecs.append(x_vec)
    
    baseplate_points = np.array(mpmt.get_xy_points('design'),dtype=np.float32)
    plt_baseplate = k3d.lines(baseplate_points, indices_mpmt, indices_type='segment', color=color_mpmt)
    plot += plt_baseplate
    
    feedthrough_points = np.array(mpmt.get_xy_points('design', feature='feedthrough'),dtype=np.float32)
    plt_feedthrough = k3d.lines(feedthrough_points, indices_ft, indices_type='segment', color=color_mpmt)
    plot += plt_feedthrough
    
    if i_mpmt%5 == 0:
        for i_pmt, pmt in enumerate(mpmt.pmts):

            # k3d expects ndarray of float32 for segments
            circle_points = np.array(pmt.get_circle_points(n_point_pmt,'design'),dtype=np.float32)
            plt_circle = k3d.lines(circle_points, indices_pmt, indices_type='segment', color=color_pmt)
            plot += plt_circle
            
        for i_led, led in enumerate(mpmt.leds):
            location, direction_x, direction_z = led.get_orientation('design')
            z_vec = list(np.array(direction_z)*led_vec_length)
            led_origins.append(location)
            led_z_vecs.append(z_vec)
    
    # k3d complains about the following not being float32!
    #plt_text = k3d.text(str(i_mpmt), position=location, reference_point='cc', size=1., label_box=False, color=color_mpmt)
    #plot += plt_text
    
#plt_z_vecs = k3d.vectors(origins=origins, vectors=z_vecs, color=color_z, head_size=250.)
#plot += plt_z_vecs

#plot led locations and axis
led_locations = np.array(led_origins, dtype=np.float32)    
plt_leds = k3d.points(positions=led_locations,
                        point_size=8.,
                        shader='3d',
                        color=color_led)
plot += plt_leds

plt_led_z_vecs = k3d.vectors(origins=led_origins, vectors=led_z_vecs, color=color_z, head_size=50.)
plot += plt_led_z_vecs

plot.display()